In [0]:
# %run ../00_config

In [0]:
print("コンテンツマスタのCSVファイルを作成します...")

In [0]:
from pyspark.sql import Row
from pyspark.sql.functions import col
import random

# ============================================================
# テーブル: ife_contents（機内エンタメコンテンツマスタ）サンプル
# ============================================================
# ---------- 画像パスデータを作成する関数 ----------
def make_img_url(file_name: str) -> str:
    """画像パスを組み立て"""
    return f"/Volumes/{MY_CATALOG}/{MY_SCHEMA}/{MY_VOLUME_CONTNETS}/{file_name}"

# ---------- 固定リスト ----------
contents = [
    # 映画、ドラマ (1〜20)
    {"content_id": i, "content_category": "映画、ドラマ", "content_img_url": make_img_url(f"{i}.png")}
    for i in range(1, 10)
] + [
    # 観光情報 (11〜13)
    {"content_id": 11, "content_category": "観光情報", "content_img_url": make_img_url("11.png")},
    {"content_id": 12, "content_category": "観光情報", "content_img_url": make_img_url("12.png")},
    {"content_id": 13, "content_category": "観光情報", "content_img_url": make_img_url("13.png")},
    # 食事・グルメ情報 (14〜16)
    {"content_id": 14, "content_category": "食事・グルメ情報", "content_img_url": make_img_url("14.png")},
    {"content_id": 15, "content_category": "食事・グルメ情報", "content_img_url": make_img_url("15.png")},
    {"content_id": 16, "content_category": "食事・グルメ情報", "content_img_url": make_img_url("16.png")},
    # 移送手段や便利な情報 (17〜19)
    {"content_id": 17, "content_category": "移送手段や便利な情報", "content_img_url": make_img_url("17.png")},
    {"content_id": 18, "content_category": "移送手段や便利な情報", "content_img_url": make_img_url("18.png")},
    {"content_id": 19, "content_category": "移送手段や便利な情報", "content_img_url": make_img_url("19.png")},
    # リラックスできるコンテンツ (20〜22)
    {"content_id": 20, "content_category": "リラックスできるコンテンツ", "content_img_url": make_img_url("20.png")},
    {"content_id": 21, "content_category": "リラックスできるコンテンツ", "content_img_url": make_img_url("21.png")},
    {"content_id": 22, "content_category": "リラックスできるコンテンツ", "content_img_url": make_img_url("22.png")},
    # 文化やマナーに関する情報 (23〜25)
    {"content_id": 23, "content_category": "文化やマナーに関する情報", "content_img_url": make_img_url("23.png")},
    {"content_id": 24, "content_category": "文化やマナーに関する情報", "content_img_url": make_img_url("24.png")},
    {"content_id": 25, "content_category": "文化やマナーに関する情報", "content_img_url": make_img_url("25.png")}
]

# ---------- 行レコードを組み立て ----------
rows = []
for content in contents:
    # duration_secはカテゴリごとにそれっぽい乱数を割り当て
    if content["content_category"] == "映画、ドラマ":
        duration_sec = random.randint(90*60, 120*60)  # 90〜120分
    else:
        duration_sec = random.randint(5*60, 15*60)    # 5〜15分

    rows.append(
        Row(
            content_id       = content["content_id"],
            content_category = content["content_category"],
            duration_sec     = duration_sec,
            content_img_url  = content["content_img_url"]
        )
    )

# ---------- DataFrame 化 ----------
df = spark.createDataFrame(rows)

# ---------- 動作確認 ----------
print("コンテンツレコード数:", df.count())      # 例: 30本
# df.show(10, truncate=False)
df.printSchema()
# print("カテゴリごとの作品数")
# df.groupBy("content_category").count().orderBy("content_category").show(truncate=False)

# # ---------- Delta テーブル保存 ----------
# df.write.format("delta")\
#   .option("comment", "機内エンタメコンテンツマスタ")\
#   .mode("overwrite")\
#   .saveAsTable(f"{MY_CATALOG}.{MY_SCHEMA}.bz_ife_contents")

# ---------- CSV 出力 ----------
out_path = f"/Volumes/{MY_CATALOG}/{MY_SCHEMA}/{MY_VOLUME}/ife_contents/ife_contents.csv"
df.coalesce(1).toPandas().to_csv(out_path, index=False)
print(out_path)

In [0]:
print("コンテンツマスタのCSVファイルを作成しました！")